# <b>機械学習の基礎-4</b>
EDA、前処理、特徴量エンジニアリング等の機械学習の一連のプロセスについてメモしておく。  

本ドキュメントの参考資料：  
* [Udemy講座【本番編!!】米国データサイエンティストがやさしく教える機械学習超入門【Pythonで実践】](https://www.udemy.com/share/108nHI3@xpAI18mdRm1C_i4744C1DGbEtA6OMBG1WO06UV5L6j73HgS7l7uap7-gtqM2l5bf/)
* [Kaggleで勝つデータ分析の技術](https://www.amazon.co.jp/gp/product/B07YTDBC3Z/ref=ppx_yo_dt_b_d_asin_title_o00?ie=UTF8&psc=1)

## 機械学習の一連の流れ
大体は下記のような流れで進んでいく。  
必ずしもこれ通りではない。  
EDAとデータクリーニング、前処理（欠損値対応の部分）は通常のデータ分析業務でも大事。    

1. EDA
2. データクリーニング
3. 前処理
4. 特徴量エンジニアリング
5. 特徴量選択
6. ハイパーパラメータチューニング
7. アンサンブル
8. テストデータに対する評価  
   
(3.～6.は適宜繰り返す。単一のモデルに対してだけでなく、複数のモデルに対して行う。)

## EDA(Exploratory data analysis、探索的データ分析)
学習データのデータ構造、統計量の算出、欠損値・外れ値の確認、可視化等を行い、データの分布や性質を理解するプロセス。  
ここで正しくデータを理解できないと正しい評価指標を選択できず、一連の作業が無駄になってしまうので非常に重要。  
いわば機械学習の作業の前提を決める部分。  

データクリーニングもEDAの作業として扱われることが多い。  
ここでは不要なカラム（重複カラム、相関係数が1、欠損が多いカラム）や不要なデータ（重複データ、欠損が多いデータ、外れ値）など、  
明らかに不要な部分や重複部分（意味合い的にも数値的にも重複しているデータ）を学習データから削除する。  

## 前処理(preprocessing)
主に下記の作業が挙げられる。  
* 欠損値対応(データクリーニングの中でやる場合も多い)
* カテゴリカル変数のエンコーディング
* 特徴量スケーリング

★重要：これら処理のfitは学習データに対してのみ行い、検証データ、テストデータにはtransformのみ行うこと！

### - 欠損値対応
学習・テストデータの欠損値に対して何らかの処理を施し、モデルが学習・予測できる状態のデータにすること。  
欠損値の対応方法として主に以下がある。  

* 欠損値のあるレコードを落とす
  * 手軽だが、貴重な学習データが減ってしまうので非推奨
* 欠損のあるカラムの代表値（平均値、中央値、最頻値等）を欠損値に代入する。
  * 手軽で簡単だが、他の特徴量との関係を考慮していないため、正確性には欠ける。
  * また、欠損値がランダムではなくなんらかの規則に従って欠損している場合は全く正しくない結果となってしまうので注意。
  * とりあえず欠損値対応してベースラインの精度出したいときなどには良いかも。
* 欠損値を新たなカテゴリとして扱う（カテゴリカラムの場合）
  * 欠損していたという情報も大事なので、カテゴリカラムの場合は基本的にこの方法が良さそう。
* 欠損があるデータ以外を学習データとして、欠損値を予測して代入する。
  * 詳細はML_notebook-16のkNNによる予測を参照。予測に使うモデルはkNNでなくてもOK。  
  欠損がないレコードを学習データ（目的変数は欠損値を予測したいカラム。特徴量は欠損値があるカラム以外）とする。  

欠損値対応する際は下記を留意すること。
* なぜ欠損があるのかを考えて対応方法を選ぶこと。
  * ランダムネスがあるのか
  * 人為的なものかシステム的なものか
* 欠損していたという情報も大事なので、欠損値の対応の必要がないモデル(GBDTなど)では必ずしも対応する必要はない。  
  基本的にGBDTではベースラインの学習では対応しない。
* 欠損値にも様々な値があることに注意("", ?, 0, -, N/A, NaN, nan)
* 欠損値を代入しても欠損があったことを表すフラグを残しておくのも学習に有効な場合がある。
* 交差検証を行う際、欠損値を代表値で埋める作業は交差検証の中で行うこと。  
  * すなわち交差検証における各Foldの学習データの代表値を算出し、それを検証データの欠損値を埋めるのに使うのが適切。  
  全学習データを使って代表値を求めて欠損値を埋めてから交差検証を行うと、検証データの情報が学習データに漏れている(leakage)ことになってしまい、  
  交差検証による評価が不当に高くなってしまう恐れがある。  
  そのため、交差検証の各分割において学習データセットから代表値を計算し、その値を学習データと検証データの両方に適用することが望ましい。  
  また、カテゴリ変数に欠損がある場合、基本的に欠損値対応後にダミー変数(One-Hot Encoding)に変換するべきであるから、これも交差検証の中で行う必要がある。  


### - カテゴリカル変数(質的変数)のエンコーディング
カテゴリカル変数のエンコーディング方法は主に以下。  

* Label Encoding
  * カテゴリごとに0から順に数字を割り当てる。  
  大中小のような順序尺度に対して用いる。
  <br></br>
* One-hot Encoding
  * 0, 1のダミー変数に変換するやつ。順序尺度・非順序尺度どちらにも使える。
  * カテゴリの数だけカラムが増えるので、カテゴリ数が多い特徴量に対しては適用を避けるべき。  
  * 線形モデルを使用する場合は多重共線性を考慮してダミー変数を一つ落とすこと。
  <br></br>
* Target Encoding (TS:target statisticsともいう)
  * 目的変数の統計量（主に平均）をカテゴリとして代用する。（もしくは新たな特徴量として使う。）
    * 例）全データ数100、色を表すカテゴリ変数が青で、目的変数が1（陽性）となる数が20のとき、20/100=0.2をラベルとして使う。  
    あるカテゴリのときに目的変数が陽性となる割合をラベルにしているとも言える。  
    多クラスの場合についてはML_notebook-18(Various_Encoding).ipynbを参照。  
  * 目的変数が数値の場合も同様に平均をとる。
  * カテゴリ変数に目的変数の情報を含めることができるので、One-hotなどよりも精度が高めになることが多いらしい。
  * 順序尺度・非順序尺度どちらにも使え、One-Hot Encodingのように特徴量が増えることはないが、  
    目的変数の情報を使っているので検証データを含めてエンコーディングしないように注意。（leakageになる）  
    よってCVの際は、各foldでの学習データを基にfit,transformし、検証データではtransformだけすること。
  * 最終的なテストデータに対するエンコーディングを行う際、手持ちの学習データ全てをfitするか、cv時の各foldで求めた平均値を使用して  
    エンコーディングする。
    <br></br>
* Embeding Encoding
  * ニューラルネットのembedding layerを使って学習する。
  * 意味合い的に近いカテゴリ同士を数値的に近いものとして扱うことができる。
  * 使用頻度はそんなに高くない。

特徴量の特性に応じてそれぞれ使い分けること。  
例えば、順序尺度ならLabel Encoding、順序尺度でないかつカテゴリ数の少ない特徴量ならOne-hot Encoding、  
どちらにも当てはまりそうにないならTarget Encodingなど。  
決定木やGBDTの場合、順序尺度でなくてもLabel Encodingを使ってOK.

交差検証する場合、基本的に交差検証の前にエンコーディングはやっておくべき。  
でないと、学習データにあるクラスが検証データにはない（またはその逆）といったことが起こって予測時にエラーになる。  
しかし、カテゴリ変数に欠損があり、欠損値を代表値等で埋めて交差検証を行う場合はエンコーディングも交差検証の中で行う必要があることに注意。  


## 特徴量エンジニアリング
既にある特徴量から、予測に有益そうな特徴量を作り出すこと。  
前処理やハイパーパラメータチューニングに時間をかけるよりも、  
特徴量エンジニアリングに時間をかけた方が精度を高められることが多く、非常に重要なプロセスである。  
下記のように色々な観点で作ってみてCVで検証してみるとよい。  
* 日時情報から年、月、日、曜日、時、分、秒を切り出す等
  * 現場では時刻データが付与されていることが多いため、よく使われる特徴量エンジニアリング手法の一つ。  
  * 時系列データに限らず有効。  
  * データの性質によっては注意が必要な場合がある。
    * 例１）テストデータ2017年以降で学習データが2017年以前の場合。外挿することになり、精度が落ちる恐れがある。  
      →2017年以降も2017年以前のyearに置き換える（同じ傾向と仮定する）か、yearを使わずに学習する必要あり。
<br></br>
* 多項式特徴量（${X_1}^2,X_1X_2$など）
<br></br>
* 数値をある範囲でbinningする（数値→カテゴリ）
<br></br>
* 四則演算（例えば、利益=売上-経費）
  * ある特徴量に対して、別の特徴量や統計量との四則演算結果を新たな特徴量として生成する。
    * カテゴリカル変数同士の結合も有効である。例えば、'blue'+'good'='blue_good'など。  
      結合した特徴量に対してさらにtarget encodingを行うことにより、結合前よりも細分化したエンコーディングができる。  
      この場合”blueかつgood”のカテゴリに対してtarget encodingをすることになり、カテゴリが細分化されるため。  
      このとき、レコード数が少なすぎるとencoding後の値(目的変数の平均値)が異常になり、過小・過大評価してしまうので注意。  
  * 特徴量の全組み合わせについてやる必要はなく、ドメイン的に意味のある特徴量を生成することが有効
    * 例）利益 = 売上 - 経費
  * 決定木ベースのモデルのように特徴量間の関係を捉えることができるモデルでも演算結果を明示的に学習させることで、  
  精度が上がることがある。（決定木の気持ちになって考える。）
<br></br>
* 集約値
  * 例えば、顧客テーブル（顧客IDが主キー）と取引記録テーブル（取引IDが主キー）があるとき、  
    顧客一人に対して取引は複数は紐づく。このとき、紐づく取引の取引金額の合計を顧客に紐づけることで、  
    新たな特徴量を生成できる。このように1:Nの関係のある２つのテーブルあるいはデータを集約して特徴量として扱うことができる。  
    集約値はmin,max,sum,mean,median,stdなど。  
    
* ドメイン的に意味のある特徴量
  * 例えば過去のデータセットには特徴量A,B,Cがあり、互いにドメイン的に関係のある特徴であるとする。  
    一方で、タスクのために学習に使うデータセットにはA,Bはあるが、Cがないとする。  
    このとき、前者のデータセットでA,BからCを予測するモデルを学習させておき、後者のA,Bに適用することで、Cを作成することができる。  
    この手法は欠損値対応にも有効。

## 特徴量選択
特徴量エンジニアリングを行って特徴量が増えた後は、  
出来るだけ目的変数に寄与する重要な特徴量のみ選択して学習に使用することが計算量・時間、リソースの観点で重要である。  
重要でない（ノイズとなる）特徴量を減らすことでvarianceが小さくなり、精度が向上することもある。  
もちろん、ビジネスでは特徴量が多いほど集めるデータの種類も多くなり、コストがかかるので、特徴量がなるべく少なく精度が高いことが理想的。  
GBDTはノイズに強いので、元々の特徴量はそのまま使い、特徴量エンジニアリングで増えた分だけ取捨選択する程度で十分との意見もある（Kaggle本）。  
特徴量選択の手法は主に下記。  
* Greedy Feature Selection  
  * 下記の手順で特徴量を選択していく方法。  
  有効な特徴量を網羅的・機械的に選択できるが、計算量が非常に多くなるので注意。
    1. 各特徴量単体でモデルを学習・評価し、最高精度となった特徴量を選択する。
    2. そこへ残りの特徴量を一つだけ追加して学習・評価する。これを残りの特徴量全パターンで実施し、最高精度となった特徴量を選択する。
    3. 最高精度を更新しなくなるまで2.を繰り返す。
   <br></br>
* Recursive Feature Elimination(RFE)
  * 全ての特徴量でモデルを構築してから予測への寄与が小さい特徴量を一つずつ減らしていく方法。
  1. 全ての特徴量でモデルを学習する
  2. 予測への寄与が最も低い特徴量を削除し、残りの特徴量でモデルを学習する
      * 特徴量の予測への寄与具合は回帰係数や決定木系アルゴリズムの重要度などを使用する
  3. 2.を指定した特徴量の数になるまで実施する
      * 最終的に残す特徴量の数はハイパーパラメータのチューニングで決める。
   <br></br>
* モデルの係数や重要度から特徴量選択
  * 線形回帰の係数や決定木・GBDTの重要度(feature importances)のようにモデルによっては  
  目的変数の予測に対する各特徴量の寄与具合を確認することができる。これらを使って特徴量を選択する方法（閾値は別途定める）  
  * GBDTで特徴量選択を行う場合はこの手法が比較的よく使われている模様。
  * 手動で重要度を確認する他、下記ライブラリ・手法がある。詳細は各参考資料を参照。
    * sklearn.feature_selection.SelectFromModelクラス
    * permutation importance
    * null importance
    * boruta
* L1正則化項による特徴量選択(Lasso)
  * 詳細は機械学習の基礎-1.ipynb参照

## ハイパーパラメータのチューニング
人間が手動で定める学習時のパラメタ（ハイパーパラメータ）を最適化することで  
精度が向上する場合がある。ハイパーパラメータはモデルによって多種多様。  
主な方法は下記。  
* Grid Search
  * 候補となるハイパーパラメータの総当たりで最も交差検証のスコア等が良い組み合わせを探す
  * 公式ドキュメントや他の事例などを参考に、あらかじめハイパーパラメータの範囲の目星をつけておく必要あり。
    * max_depth: [5, 10, 15]など
  * 計算量が膨大になりがちなので、ハイパーパラメータの意味をしっかり理解して出来る限り不要な値は含めないこと。
  <br></br>
* Random Search
  * ランダムな組み合わせで探す。
  * パラメタがとる値の範囲を定め、任意の確率分布の下でパラメタ値を決める。リストでパラメタ値を指定してもOK。
  <br></br>
* Optimization
  * 最適化アルゴリズムを用いて最適なハイパーパラメータを探索する方法
    * Grid SearchやRandom Searchでは指定した値の範囲内で網羅的あるいはランダムに選ばれたパラメタの組み合わせを試すだけなので、  
      範囲内で良い組み合わせを見つけてもその組み合わせの近傍を重点的に評価することはできない。  
      最適化によるチューニングでは組み合わせの評価結果を踏まえて、より評価が良くなるように探索していく。
  * ベイズ最適化がよく使われる。ベイズ最適化の詳細化は下に記す。

### ベイズ最適化
ハイパーパラメータの最適化で最も使われる手法の一つ。  
ベイズ最適化は真の関数が不明である場合の最適化問題（ブラックボックス最適化問題）を解く手法である。  
ブラックボックス最適化とは、入力と出力は分かっているが、入出力間の関係（真の関数）がわかっていない場合において出力を最適化する入力を求めること。  
ハイパーパラメータのチューニングはハイパーパラメータを入力、損失を出力と考えると、入出力をつなぐ関数が不明※であるため、  
ブラックボックス最適化問題とみなすことが出来る。  
※学習器は入出力をつなぐものであるが、明にハイパーパラメータと損失（および損失の勾配）の関係を表現できるものではない。(線形回帰とかは別かも)  

ベイズ最適化では、真の関数を推定した関数（獲得関数：acuisition function）を使って最適化問題を解く(出力を最小にする入力を探す)。  
大まかな流れは下記。  
$$入力x --> 真の関数f(x) -->出力y$$
1. 適当な入力$x_0$で$f(x_0)$（すなわち出力$y_0$）を得る
2. 得られた$f(x_0)$で獲得関数$f'(x)$を推定する。
3. $f'(x)$を最小化する$x_1$を求める
4. $x_1$で$f(x_1)$を得る
5. 得られた$f(x_1)$および$f(x_0)$で獲得関数$f'(x)$を更新
6. 3.～5.を指定した条件まで続ける。

どんどん入力と出力のセットを揃えていって、これらの関係を説明する関数を類推していくイメージ（帰納的な考え方）。  
大変そうだが、自然で理にかなっている考え方のように思う。  